In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from sklearn import model_selection
from sklearn import svm
from sklearn import metrics
from sklearn import datasets
from sklearn import preprocessing

In [ ]:
from sklearn import pipeline

In [ ]:
data = datasets.load_breast_cancer()

In [ ]:
X = data.data

In [ ]:
y = data.target

### Validacioni skup

In [ ]:
X_train_validation, X_test, y_train_validation, y_test = model_selection.train_test_split(X, y, test_size=0.3, stratify = y, random_state = 25)

In [ ]:
X_train, X_validation, y_train, y_validation = model_selection.train_test_split(X_train_validation, y_train_validation, test_size = 0.2, stratify = y_train_validation, random_state = 25)

Pravimo nizove potencijalnih optimalnih vrednosti regularizacionog parametra `C` i sirine kernela `gamma`:

In [ ]:
Cs = np.array([10**i for i in range(-5, 5)])
gammas = np.array([10**i for i in range(-3, 3)])

In [ ]:
print(Cs, gammas)

In [ ]:
scaler_tv = preprocessing.StandardScaler()
scaler_tv.fit(X_train)
X_train = scaler_tv.transform(X_train)
X_validation = scaler_tv.transform(X_validation)

In [ ]:
best_score = 0 
best_params = {'C':0, 'gamma': 0}

# za sve kombinacije C i gamma parametara
for C in Cs: 
    for gamma in gammas:
        # treniramo model na skupu za treniranje
        model = svm.SVC(C = C, gamma = gamma)
        model.fit(X_train, y_train)
        # ocenjujemo model na skupu za validaciju
        y_predicted = model.predict(X_validation)
        score = metrics.accuracy_score(y_validation, y_predicted)

        if score > best_score: 
            best_score = score
            best_params['C'] = C
            best_params['gamma'] = gamma

In [ ]:
best_score

In [ ]:
best_params

In [ ]:
scaler_tt = preprocessing.StandardScaler()
scaler_tt.fit(X_train_validation)
X_train_validation = scaler_tt.transform(X_train_validation)
X_test = scaler_tt.transform(X_test)

In [ ]:
model = svm.SVC(C=best_params['C'], gamma=best_params['gamma'])

In [ ]:
model.fit(X_train_validation, y_train_validation)

In [ ]:
y_predicted = model.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_predicted)

### Unakrsna validacija za evaluaciju modela sa parametrima

In [ ]:
model = svm.SVC(C=best_params['C'], gamma=best_params['gamma'])

In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
svc_pipeline =  pipeline.make_pipeline(scaler, model)

In [ ]:
scores = model_selection.cross_val_score(svc_pipeline, X, y, scoring='accuracy', cv = 10)

In [ ]:
scores

In [ ]:
scores.mean()

### Izbor vrednosti hiperparametra unakrsnom validacijom

In [ ]:
grid_model = pipeline.Pipeline(steps=[('scaler', preprocessing.StandardScaler()), ('svm', svm.SVC())])
params = {
    'svm__C': [10**i for i in range(-5, 5)], 
    'svm__gamma': [10**i for i in range(-3, 3)]
}

In [ ]:
grid = model_selection.GridSearchCV(grid_model, param_grid=params, scoring='accuracy', cv=10, return_train_score=True)

In [ ]:
grid.fit(X_train_validation, y_train_validation)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

### Finalni model

In [ ]:
model_best_grid = svm.SVC(C=best_params['C'], gamma=best_params['gamma'])

In [ ]:
model_best_grid.fit(X_train_validation, y_train_validation)

In [ ]:
y_pred = model_best_grid.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred) # ovo je vrednost koju prijavljujemo kao meru kvaliteta modela

In [ ]:
final_scaler = preprocessing.StandardScaler()
X = final_scaler.fit_transform(X)

In [ ]:
final_model = svm.SVC(C=best_params['C'], gamma=best_params['gamma']) # ovo je model koji cuvamo
final_model.fit(X, y) 

In [ ]:
import pickle

In [ ]:
with open('svm_final_model.model', 'wb')  as pickle_file:
    pickle.dump(final_model, pickle_file)